In [58]:
from datetime import date
import pandas as pd
import numpy as np
import yaml
from sqlalchemy import create_engine, inspect, text

In [59]:
# Cargar configuración
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

# Crear conexiones
url_fuente = f"postgresql://{config_fuente['user']}:{config_fuente['password']}@{config_fuente['host']}:{config_fuente['port']}/{config_fuente['dbname']}"
url_bodega = f"postgresql://{config_bodega['user']}:{config_bodega['password']}@{config_bodega['host']}:{config_bodega['port']}/{config_bodega['dbname']}"

fuente_conn = create_engine(url_fuente)
bodega_conn = create_engine(url_bodega)

In [60]:
# Consulta SQL para obtener los servicios y sus estados
query = """
SELECT 
    s.id as servicio_id,
    s.cliente_id,
    s.mensajero_id,
    es.estado_id,
    es.fecha as fecha_estado,
    es.hora as hora_estado
FROM mensajeria_servicio s
JOIN mensajeria_estadosservicio es ON s.id = es.servicio_id
ORDER BY s.id, es.fecha, es.hora
"""

In [61]:
# Leer datos de la fuente
df = pd.read_sql(query, fuente_conn)
dim_fecha = pd.read_sql_table('dim_fecha', bodega_conn)
dim_cliente = pd.read_sql_table('dim_cliente', bodega_conn)
dim_mensajero = pd.read_sql_table('dim_mensajero', bodega_conn)
dim_estado = pd.read_sql_table('dim_estado', bodega_conn)

In [62]:
# Limpiar formato de hora
def limpiar_hora(hora_str):
    """
    Limpia el formato de hora eliminando los milisegundos si existen
    """
    try:
        # Si la hora tiene milisegundos (contiene un punto)
        if '.' in str(hora_str):
            return str(hora_str).split('.')[0]
        return str(hora_str)
    except:
        return None

# Limpiar el formato de hora
df['hora_estado'] = df['hora_estado'].apply(limpiar_hora)

In [63]:
print(df['hora_estado'].head())

0    16:22:18
1    17:51:20
2    12:02:48
3    12:16:00
4    17:07:55
Name: hora_estado, dtype: object


In [64]:
print("Tipo de dato fecha_estado:", df['fecha_estado'].dtype)
print("Tipo de dato hora_estado:", df['hora_estado'].dtype)
print("\nEjemplos de fecha_estado:")
print(df['fecha_estado'].head())
print("\nEjemplos de hora_estado:")
print(df['hora_estado'].head())

Tipo de dato fecha_estado: object
Tipo de dato hora_estado: object

Ejemplos de fecha_estado:
0    2023-09-19
1    2023-10-13
2    2023-10-31
3    2023-10-31
4    2023-10-31
Name: fecha_estado, dtype: object

Ejemplos de hora_estado:
0    16:22:18
1    17:51:20
2    12:02:48
3    12:16:00
4    17:07:55
Name: hora_estado, dtype: object


In [65]:
# Pivotear los datos
df_pivot = pd.pivot_table(
    df,
    index=['servicio_id', 'cliente_id', 'mensajero_id'],
    columns='estado_id',
    values=['fecha_estado', 'hora_estado'],
    aggfunc='first'
).reset_index()

In [66]:
# Aplanar los nombres de las columnas multinivel
df_pivot.columns = ['servicio_id', 'cliente_id', 'mensajero_id'] + [
    f"{col[0]}_{estado}" for col in df_pivot.columns[3:]
    for estado in ['iniciado' if col[1]==1 else 
                   'asignado' if col[1]==2 else 
                   'recogido' if col[1]==4 else 
                   'entregado' if col[1]==5 else 
                   'cerrado' if col[1]==6 else str(col[1])]
]

In [67]:
# Renombrar las columnas para mayor claridad
for estado in ['iniciado', 'asignado', 'recogido', 'entregado', 'cerrado']:
    df_pivot = df_pivot.rename(columns={
        f'fecha_estado_{estado}': f'fecha_{estado}',
        f'hora_estado_{estado}': f'hora_{estado}'
    })

In [68]:
# Después del pivoteo, separar fecha y hora
def separar_fecha_hora(timestamp):
    if pd.isna(timestamp):
        return pd.Series({'fecha': None, 'hora': None})
    return pd.Series({'fecha': timestamp.date(), 'hora': timestamp.strftime('%H:%M:%S')})

In [69]:
# Función modificada para calcular tiempos entre estados
def calcular_tiempo_entre_estados(fecha1, hora1, fecha2, hora2):
    try:
        if pd.isna(fecha1) or pd.isna(fecha2) or pd.isna(hora1) or pd.isna(hora2):
            return None
        
        timestamp1 = pd.to_datetime(f"{fecha1} {hora1}")
        timestamp2 = pd.to_datetime(f"{fecha2} {hora2}")
        
        segundos = (timestamp2 - timestamp1).total_seconds()
        if segundos < 0:
            return None
            
        hours = int(segundos // 3600)
        minutes = int((segundos % 3600) // 60)
        seconds = int(segundos % 60)
        return f"{hours:02d}:{minutes:02d}:{seconds:02d}"
    except:
        return None

In [70]:
# Calcular los tiempos
df_pivot['tiempo_asignacion'] = df_pivot.apply(
    lambda row: calcular_tiempo_entre_estados(
        row['fecha_iniciado'], 
        row['hora_iniciado'],
        row['fecha_asignado'],
        row['hora_asignado']
    ), axis=1
)

df_pivot['tiempo_recogida'] = df_pivot.apply(
    lambda row: calcular_tiempo_entre_estados(
        row['fecha_asignado'], 
        row['hora_asignado'],
        row['fecha_recogido'],
        row['hora_recogido']
    ), axis=1
)

df_pivot['tiempo_entrega'] = df_pivot.apply(
    lambda row: calcular_tiempo_entre_estados(
        row['fecha_recogido'], 
        row['hora_recogido'],
        row['fecha_entregado'],
        row['hora_entregado']
    ), axis=1
)

df_pivot['tiempo_cierre'] = df_pivot.apply(
    lambda row: calcular_tiempo_entre_estados(
        row['fecha_entregado'], 
        row['hora_entregado'],
        row['fecha_cerrado'],
        row['hora_cerrado']
    ), axis=1
)

In [72]:
# Convertir las fechas al mismo formato
print("\nTipos de datos antes de la conversión:")
print("fecha_iniciado dtype:", df_pivot['fecha_iniciado'].dtype)
print("fecha en dim_fecha dtype:", dim_fecha['fecha'].dtype)

# Convertir fecha_iniciado a datetime
df_pivot['fecha_iniciado'] = pd.to_datetime(df_pivot['fecha_iniciado']).dt.date
dim_fecha['fecha'] = pd.to_datetime(dim_fecha['fecha']).dt.date

print("\nTipos de datos después de la conversión:")
print("fecha_iniciado dtype:", df_pivot['fecha_iniciado'].dtype)
print("fecha en dim_fecha dtype:", dim_fecha['fecha'].dtype)


Tipos de datos antes de la conversión:
fecha_iniciado dtype: object
fecha en dim_fecha dtype: datetime64[ns]

Tipos de datos después de la conversión:
fecha_iniciado dtype: object
fecha en dim_fecha dtype: object


In [73]:
# Realizar los merges con las dimensiones
hecho_acumulado = df_pivot.merge(
    dim_fecha[['key_dim_fecha', 'fecha']], 
    left_on='fecha_iniciado', 
    right_on='fecha',
    how='left'
)

In [74]:
hecho_acumulado = hecho_acumulado.merge(
    dim_cliente[['key_dim_cliente', 'cliente_id']], 
    on='cliente_id',
    how='left'
)

In [75]:
hecho_acumulado = hecho_acumulado.merge(
    dim_mensajero[['key_dim_mensajero', 'mensajero_id']], 
    on='mensajero_id',
    how='left'
)

In [76]:
# Seleccionar columnas finales
columnas_finales = [
    'servicio_id',
    'key_dim_fecha',
    'key_dim_cliente',
    'key_dim_mensajero',
    'fecha_iniciado',
    'hora_iniciado',
    'fecha_asignado',
    'hora_asignado',
    'fecha_recogido',
    'hora_recogido',
    'fecha_entregado',
    'hora_entregado',
    'fecha_cerrado',
    'hora_cerrado',
    'tiempo_asignacion',
    'tiempo_recogida',
    'tiempo_entrega',
    'tiempo_cierre'
]

In [77]:
hecho_acumulado = hecho_acumulado[columnas_finales]
# Agregar fecha de carga
hecho_acumulado['saved'] = date.today()

# Verificaciones

In [78]:
# Verificaciones
print("\nInformación del DataFrame:")
print(hecho_acumulado.info())


Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27703 entries, 0 to 27702
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   servicio_id        27703 non-null  int64  
 1   key_dim_fecha      27702 non-null  float64
 2   key_dim_cliente    27703 non-null  int64  
 3   key_dim_mensajero  27703 non-null  int64  
 4   fecha_iniciado     27702 non-null  object 
 5   hora_iniciado      27702 non-null  object 
 6   fecha_asignado     27702 non-null  object 
 7   hora_asignado      27702 non-null  object 
 8   fecha_recogido     27016 non-null  object 
 9   hora_recogido      27016 non-null  object 
 10  fecha_entregado    26952 non-null  object 
 11  hora_entregado     26952 non-null  object 
 12  fecha_cerrado      8290 non-null   object 
 13  hora_cerrado       8290 non-null   object 
 14  tiempo_asignacion  27692 non-null  object 
 15  tiempo_recogida    27014 non-null  object 

In [79]:
hecho_acumulado.head()

,servicio_id,key_dim_fecha,key_dim_cliente,key_dim_mensajero,fecha_iniciado,hora_iniciado,fecha_asignado,hora_asignado,fecha_recogido,hora_recogido,fecha_entregado,hora_entregado,fecha_cerrado,hora_cerrado,tiempo_asignacion,tiempo_recogida,tiempo_entrega,tiempo_cierre,saved
0,7,261.0,7,0,2023-09-19,16:22:18,2023-10-13,17:51:20,2023-10-31,12:02:48,2023-10-31,17:07:55,2023-10-31,12:16:00,577:29:02,426:11:28,05:05:07,None,2024-11-08
1,8,261.0,7,13,2023-09-19,16:30:05,2023-12-20,20:14:43,2024-02-14,15:34:18,2024-04-09,16:08:35,NaN,NaN,2211:44:38,1339:19:35,1320:34:17,None,2024-11-08
2,9,261.0,7,13,2023-09-19,16:30:05,2023-12-28,19:33:01,NaN,NaN,NaN,NaN,NaN,NaN,2403:02:56,None,None,None,2024-11-08
3,10,261.0,7,13,2023-09-19,16:35:52,2023-12-28,19:33:07,2024-02-18,00:21:47,2024-03-10,09:58:27,NaN,NaN,2402:57:15,1228:48:40,513:36:40,None,2024-11-08
4,11,261.0,7,13,2023-09-19,16:37:54,2023-12-09,13:13:59,2024-01-31,10:29:55,NaN,NaN,NaN,NaN,1940:36:05,1269:15:56,None,None,2024-11-08


In [80]:
print("\nVerificación de tiempos calculados:")
print(hecho_acumulado[['tiempo_asignacion', 'tiempo_recogida', 'tiempo_entrega', 'tiempo_cierre']].head())


Verificación de tiempos calculados:
  tiempo_asignacion tiempo_recogida tiempo_entrega tiempo_cierre
0         577:29:02       426:11:28       05:05:07          None
1        2211:44:38      1339:19:35     1320:34:17          None
2        2403:02:56            None           None          None
3        2402:57:15      1228:48:40      513:36:40          None
4        1940:36:05      1269:15:56           None          None


In [81]:
print("\nRegistros con tiempos nulos:")
for tiempo in ['tiempo_asignacion', 'tiempo_recogida', 'tiempo_entrega', 'tiempo_cierre']:
    nulos = hecho_acumulado[tiempo].isna().sum()
    print(f"{tiempo}: {nulos} registros nulos")


Registros con tiempos nulos:
tiempo_asignacion: 11 registros nulos
tiempo_recogida: 689 registros nulos
tiempo_entrega: 752 registros nulos
tiempo_cierre: 19415 registros nulos


In [82]:
print("\nVerificar registros con fechas fuera de secuencia:")
for idx, row in hecho_acumulado.iterrows():
    fechas = [
        (pd.to_datetime(f"{row['fecha_iniciado']} {row['hora_iniciado']}") if pd.notna(row['fecha_iniciado']) else None, 'iniciado'),
        (pd.to_datetime(f"{row['fecha_asignado']} {row['hora_asignado']}") if pd.notna(row['fecha_asignado']) else None, 'asignado'),
        (pd.to_datetime(f"{row['fecha_recogido']} {row['hora_recogido']}") if pd.notna(row['fecha_recogido']) else None, 'recogido'),
        (pd.to_datetime(f"{row['fecha_entregado']} {row['hora_entregado']}") if pd.notna(row['fecha_entregado']) else None, 'entregado'),
        (pd.to_datetime(f"{row['fecha_cerrado']} {row['hora_cerrado']}") if pd.notna(row['fecha_cerrado']) else None, 'cerrado')
    ]
    fechas = [(f, e) for f, e in fechas if f is not None]
    for i in range(len(fechas)-1):
        if fechas[i][0] > fechas[i+1][0]:
            print(f"\nServicio {row['servicio_id']} tiene {fechas[i][1]} después de {fechas[i+1][1]}")
            print(f"{fechas[i][1]}: {row[f'fecha_{fechas[i][1]}']} {row[f'hora_{fechas[i][1]}']}")
            print(f"{fechas[i+1][1]}: {row[f'fecha_{fechas[i+1][1]}']} {row[f'hora_{fechas[i+1][1]}']}")



Verificar registros con fechas fuera de secuencia:

Servicio 7 tiene entregado después de cerrado
entregado: 2023-10-31 17:07:55
cerrado: 2023-10-31 12:16:00

Servicio 22 tiene iniciado después de asignado
iniciado: 2023-09-23 19:19:28
asignado: 2023-09-22 19:45:37

Servicio 23 tiene iniciado después de asignado
iniciado: 2023-09-23 19:19:58
asignado: 2023-09-22 19:45:37

Servicio 24 tiene iniciado después de asignado
iniciado: 2023-09-23 19:22:13
asignado: 2023-09-22 19:45:37

Servicio 25 tiene iniciado después de asignado
iniciado: 2023-09-23 19:22:24
asignado: 2023-09-22 19:45:30

Servicio 26 tiene iniciado después de asignado
iniciado: 2023-09-23 19:25:56
asignado: 2023-09-22 19:45:20

Servicio 83 tiene iniciado después de asignado
iniciado: 2024-01-04 19:07:55
asignado: 2024-01-03 22:20:44

Servicio 86 tiene iniciado después de asignado
iniciado: 2024-01-04 19:31:08
asignado: 2024-01-03 19:31:51

Servicio 87 tiene iniciado después de asignado
iniciado: 2024-01-04 22:11:13
asignad

In [83]:
# Guardar en la bodega
hecho_acumulado.to_sql('hecho_entrega_acumulado', bodega_conn, if_exists='replace', index_label='key_hecho_entrega_acumulado')

703

In [ ]:
#Pregunta 5: Mensajeros más eficientes (Los que más servicios prestan)
"""
WITH tiempo_total_servicio AS (
    SELECT 
        h.key_dim_mensajero,
        h.servicio_id,
        -- Convertir los tiempos de formato HH:MM:SS a minutos para poder sumarlos
        EXTRACT(HOUR FROM tiempo_asignacion::interval) * 60 + 
        EXTRACT(MINUTE FROM tiempo_asignacion::interval) +
        EXTRACT(HOUR FROM tiempo_recogida::interval) * 60 + 
        EXTRACT(MINUTE FROM tiempo_recogida::interval) +
        EXTRACT(HOUR FROM tiempo_entrega::interval) * 60 + 
        EXTRACT(MINUTE FROM tiempo_entrega::interval) +
        EXTRACT(HOUR FROM tiempo_cierre::interval) * 60 + 
        EXTRACT(MINUTE FROM tiempo_cierre::interval) as tiempo_total_minutos
    FROM hecho_entrega_acumulado h
)
SELECT 
    m.mensajero_id,
    COUNT(t.servicio_id) as total_servicios,
    AVG(t.tiempo_total_minutos) as promedio_minutos_por_servicio,
    AVG(t.tiempo_total_minutos)/60 as promedio_horas_por_servicio
FROM tiempo_total_servicio t
JOIN dim_mensajero m ON t.key_dim_mensajero = m.key_dim_mensajero
GROUP BY m.mensajero_id
ORDER BY total_servicios DESC;
"""

In [ ]:
#Pregunta 7: Cuál es el tiempo promedio de entrega desde que se solicita el servicio hasta que se cierra el caso
'''
WITH tiempos_totales AS (
    SELECT 
        servicio_id,
        fecha_iniciado || ' ' || hora_iniciado as inicio,
        fecha_cerrado || ' ' || hora_cerrado as fin
    FROM hecho_entrega_acumulado
    WHERE fecha_iniciado IS NOT NULL 
    AND fecha_cerrado IS NOT NULL
)
SELECT 
    COUNT(*) as total_servicios,
    AVG(EXTRACT(EPOCH FROM (fin::timestamp - inicio::timestamp))/60) as promedio_minutos,
    AVG(EXTRACT(EPOCH FROM (fin::timestamp - inicio::timestamp))/3600) as promedio_horas,
    AVG(EXTRACT(EPOCH FROM (fin::timestamp - inicio::timestamp))/86400) as promedio_dias
FROM tiempos_totales;
'''

In [ ]:
#Pregunta 8: Mostrar los tiempos de espera por cada fase del servicio y en qué fase hay más demoras
'''
WITH tiempos_por_fase AS (
    SELECT 
        'Asignación' as fase,
        tiempo_asignacion as tiempo,
        EXTRACT(HOUR FROM tiempo_asignacion::interval) * 60 + 
        EXTRACT(MINUTE FROM tiempo_asignacion::interval) as minutos
    FROM hecho_entrega_acumulado
    WHERE tiempo_asignacion IS NOT NULL
    
    UNION ALL
    
    SELECT 
        'Recogida' as fase,
        tiempo_recogida as tiempo,
        EXTRACT(HOUR FROM tiempo_recogida::interval) * 60 + 
        EXTRACT(MINUTE FROM tiempo_recogida::interval) as minutos
    FROM hecho_entrega_acumulado
    WHERE tiempo_recogida IS NOT NULL
    
    UNION ALL
    
    SELECT 
        'Entrega' as fase,
        tiempo_entrega as tiempo,
        EXTRACT(HOUR FROM tiempo_entrega::interval) * 60 + 
        EXTRACT(MINUTE FROM tiempo_entrega::interval) as minutos
    FROM hecho_entrega_acumulado
    WHERE tiempo_entrega IS NOT NULL
    
    UNION ALL
    
    SELECT 
        'Cierre' as fase,
        tiempo_cierre as tiempo,
        EXTRACT(HOUR FROM tiempo_cierre::interval) * 60 + 
        EXTRACT(MINUTE FROM tiempo_cierre::interval) as minutos
    FROM hecho_entrega_acumulado
    WHERE tiempo_cierre IS NOT NULL
)
SELECT 
    fase,
    COUNT(*) as total_servicios,
    AVG(minutos) as promedio_minutos,
    MIN(tiempo) as tiempo_minimo,
    MAX(tiempo) as tiempo_maximo,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY minutos) as mediana_minutos
FROM tiempos_por_fase
GROUP BY fase
ORDER BY promedio_minutos DESC;
'''